# Assignment 2 DSC 102 SP25

## Introduction

In this assignment we will conduct data engineering for the Amazon dataset. It is divided into 2 parts. The extracted features in Part 1 will be used for the Part 2 of assignment, where you train a model (or models) to predict user ratings for a product.

We will be using Apache Spark for this assignment. The default Spark API will be DataFrame, as it is now the recommended choice over the RDD API. That being said, please feel free to switch back to the RDD API if you see it as a better fit for the task. We provide you an option to request RDD format to start with. Also you can switch between DataFrame and RDD in your solution. 

Another newer API is Koalas, which is also avaliable. However, it has constraints and is not applicable to most tasks. Refer to the PA statement for detail.

### Set the following parameters

In [1]:
PID = 'A17680599' # your pid, for instance: 'a43223333'
INPUT_FORMAT = 'dataframe' # choose a format of your input data, valid options: 'dataframe', 'rdd', 'koalas'

In [2]:
# Boiler plates, do NOT modify
%load_ext autoreload
%autoreload 2
import os
import getpass
from pyspark.sql import SparkSession
from utilities import SEED
from utilities import PA2Test
from utilities import PA2Data
from utilities import data_cat
from pa2_main import PA2Executor
import time
if INPUT_FORMAT == 'dataframe':
    import pyspark.ml as M
    import pyspark.sql.functions as F
    import pyspark.sql.types as T
if INPUT_FORMAT == 'koalas':
    import databricks.koalas as ks
elif INPUT_FORMAT == 'rdd':
    import pyspark.mllib as M
    from pyspark.mllib.feature import Word2Vec
    from pyspark.mllib.linalg import Vectors
    from pyspark.mllib.linalg.distributed import RowMatrix

os.environ['PYSPARK_SUBMIT_ARGS'] = '--py-files utilities.py,assignment2.py \
--deploy-mode client \
pyspark-shell'

class args:
    review_filename = data_cat.review_filename
    product_filename = data_cat.product_filename
    product_processed_filename = data_cat.product_processed_filename
    ml_features_train_filename = data_cat.ml_features_train_filename
    ml_features_test_filename = data_cat.ml_features_test_filename
    output_root = '/home/{}/{}-pa2/test_results'.format(getpass.getuser(), PID)
    test_results_root = data_cat.test_results_root
    pid = PID

pa2 = PA2Executor(args, input_format=INPUT_FORMAT)
data_io = pa2.data_io
data_dict = pa2.data_dict
begin = time.time()


25/05/29 02:03:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/opt/bitnami/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Loading datasets ...

Done


In [3]:
# Import your own dependencies
from pyspark.ml.feature import Word2Vec
from pyspark.ml.stat import Summarizer
from pyspark.ml.feature import StringIndexer, OneHotEncoder, PCA
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import col, explode, mean, count, variance, when, size
from pyspark.sql.types import FloatType

#-----------------------------

# Part 1: Feature Engineering

In [4]:
# Bring the part_1 datasets to memory and de-cache part_2 datasets. 
# Execute this once before you start working on this Part
data_dict, _ = data_io.cache_switch(data_dict, 'part_1')

# Task0: warm up 
This task is provided for you to get familiar with Spark API. We will use the dataframe API to demonstrate. Solution is given to you and this task won't be graded.

Refer to https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/index.html for API guide.

The task is to implement the function below. Given the ```product_data``` table:
1. Take and print five rows.

1. Select only the ```asin``` column, then print five rows of it.

1. Select the row where ```asin = B00I8KEOTM``` and print it.

1. Count the total number of rows.

1. Calculate the mean ```price```.

1. You need to conduct the above operations, then extract some statistics out of the generated columns. You need to put the statistics in a python dictionary named ```res```. The description and schema of it are as follows:
    ```
    res
     | -- count_total: int -- count of total rows of the entire table after your operations
     | -- mean_price: float -- mean value of column price
    ```

In [5]:
def task_0(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    asin_column = 'asin'
    overall_column = 'overall'
    # Outputs:
    mean_rating_column = 'meanRating'
    count_rating_column = 'countRating'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------

    product_data.show(5)
    product_data[['asin']].show(5)
    product_data.where(F.col('asin') == 'B00I8KEOTM').show()
    count_rows = product_data.count()
    mean_price = product_data.select(F.avg(F.col('price'))).head()[0]
    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    # Calculate the values programmatically. Do not change the keys and do not
    # hard-code values in the dict. Your submission will be evaluated with
    # different inputs.
    # Modify the values of the following dictionary accordingly.
    res = {'count_total': None, 'mean_price': None}
    
    # Modify res:

    res['count_total'] = count_rows
    res['mean_price'] = mean_price

    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    return res
    # -------------------------------------------------------------------------

In [6]:
if INPUT_FORMAT == 'dataframe':
    res = task_0(data_io, data_dict['product'])
    pa2.tests.test(res, 'task_0')

+----------+--------------------+--------------------+--------------------+-----+--------------------+
|      asin|           salesRank|          categories|               title|price|             related|
+----------+--------------------+--------------------+--------------------+-----+--------------------+
|B00I8HVV6E|{Home &amp; Kitch...|[[Home & Kitchen,...|Intelligent Desig...|27.99|{also_viewed -> [...|
|B00I8KEOTM|                null|[[Apps for Androi...|                null| null|{also_viewed -> [...|
|B00I8KCW4G|{Clothing -> 2233...|[[Clothing, Shoes...|eShakti Women's P...|41.95|{also_viewed -> [...|
|B00I8JKCQW|{Clothing -> 1405...|[[Clothing, Shoes...|Lady Slimming Mid...| null|{also_viewed -> [...|
|B00I8JKI8E|{Home &amp; Kitch...|[[Clothing, Shoes...|3 Tier Bangle Bra...|24.99|{also_viewed -> [...|
+----------+--------------------+--------------------+--------------------+-----+--------------------+
only showing top 5 rows

+----------+
|      asin|
+----------+
|B00I8HVV

+----------+---------+--------------------+-----+-----+--------------------+
|      asin|salesRank|          categories|title|price|             related|
+----------+---------+--------------------+-----+-----+--------------------+
|B00I8KEOTM|     null|[[Apps for Androi...| null| null|{also_viewed -> [...|
+----------+---------+--------------------+-----+-----+--------------------+



tests for task_0 --------------------------------------------------------------
2/2 passed
-------------------------------------------------------------------------------


# Task1

In [7]:
# %load -s task_1 assignment2.py
def task_1(data_io, review_data, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    asin_column = 'asin'
    overall_column = 'overall'
    # Outputs:
    mean_rating_column = 'meanRating'
    count_rating_column = 'countRating'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------

    # Aggregate review_data by asin
    review_stats = review_data.groupBy(asin_column).agg(
        F.mean(overall_column).alias(mean_rating_column),
        F.count(overall_column).alias(count_rating_column)
    )

    # Left join with product_data to include all products
    joined = product_data.select(asin_column).join(
        review_stats, on=asin_column, how='left'
    )

    # Compute stats for res dict
    count_total = joined.count()

    mean_meanRating = joined.select(F.mean(mean_rating_column)).first()[0]
    variance_meanRating = joined.select(F.variance(mean_rating_column)).first()[0]
    numNulls_meanRating = joined.filter(F.col(mean_rating_column).isNull()).count()

    mean_countRating = joined.select(F.mean(count_rating_column)).first()[0]
    variance_countRating = joined.select(F.variance(count_rating_column)).first()[0]
    numNulls_countRating = joined.filter(F.col(count_rating_column).isNull()).count()


    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    # Calculate the values programmaticly. Do not change the keys and do not
    # hard-code values in the dict. Your submission will be evaluated with
    # different inputs.
    # Modify the values of the following dictionary accordingly.
    res = {
        'count_total': None,
        'mean_meanRating': None,
        'variance_meanRating': None,
        'numNulls_meanRating': None,
        'mean_countRating': None,
        'variance_countRating': None,
        'numNulls_countRating': None
    }
    
    # Modify res:
    res = {
        'count_total': int(count_total),
        'mean_countRating': float(mean_countRating) if mean_countRating is not None else None,
        'mean_meanRating': float(mean_meanRating) if mean_meanRating is not None else None,
        'numNulls_countRating': int(numNulls_countRating),
        'numNulls_meanRating': int(numNulls_meanRating),
        'variance_countRating': float(variance_countRating) if variance_countRating is not None else None,
        'variance_meanRating': float(variance_meanRating) if variance_meanRating is not None else None
    }



    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_1')
    return res
    # -------------------------------------------------------------------------


In [8]:
res = task_1(data_io, data_dict['review'], data_dict['product'])
pa2.tests.test(res, 'task_1')

tests for task_1 --------------------------------------------------------------
Test 1/7 : count_total ... Pass
Test 2/7 : mean_countRating ... Pass
Test 3/7 : mean_meanRating ... Pass
Test 4/7 : numNulls_countRating ... Pass
Test 5/7 : numNulls_meanRating ... Pass
Test 6/7 : variance_countRating ... Pass
Test 7/7 : variance_meanRating ... Pass
7/7 passed
-------------------------------------------------------------------------------


True


# Task 2

In [9]:
# %load -s task_2 assignment2.py
def task_2(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    salesRank_column = 'salesRank'
    categories_column = 'categories'
    asin_column = 'asin'
    # Outputs:
    category_column = 'category'
    bestSalesCategory_column = 'bestSalesCategory'
    bestSalesRank_column = 'bestSalesRank'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------

    # If possible, extract category[0][0]
    category_col = F.when(
        (F.col(categories_column).isNotNull()) &
        (F.size(F.col(categories_column)) > 0) &
        (F.col(categories_column)[0].isNotNull()) &
        (F.size(F.col(categories_column)[0]) > 0) &
        (F.col(categories_column)[0][0].isNotNull()) &
        (F.col(categories_column)[0][0] != ""),
        F.col(categories_column)[0][0]
    ).otherwise(None).alias(category_column)


    # Extract bestSalesCategory and bestSalesRank from salesRank map
    best_sales_cat = F.when(
        F.col(salesRank_column).isNotNull() & (F.size(F.map_keys(F.col(salesRank_column))) > 0),
        F.map_keys(F.col(salesRank_column))[0]
    ).otherwise(None).alias(bestSalesCategory_column)

    best_sales_rank = F.when(
        F.col(salesRank_column).isNotNull() & (F.size(F.map_values(F.col(salesRank_column))) > 0),
        F.map_values(F.col(salesRank_column))[0]
    ).otherwise(None).alias(bestSalesRank_column)

    # Add new columns
    df = product_data.select(
        F.col(asin_column),
        category_col,
        best_sales_cat,
        best_sales_rank
    )

    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'mean_bestSalesRank': None,
        'variance_bestSalesRank': None,
        'numNulls_category': None,
        'countDistinct_category': None,
        'numNulls_bestSalesCategory': None,
        'countDistinct_bestSalesCategory': None
    }
    # Modify res:

    res = {
        'count_total': int(df.count()),
        'mean_bestSalesRank': float(df.select(F.mean(bestSalesRank_column)).first()[0]) \
            if df.select(F.mean(bestSalesRank_column)).first()[0] is not None else None,
        'variance_bestSalesRank': float(df.select(F.variance(bestSalesRank_column)).first()[0]) \
            if df.select(F.variance(bestSalesRank_column)).first()[0] is not None else None,
        'numNulls_category': int(df.filter(F.col(category_column).isNull()).count()),
        'countDistinct_category': int(df.select(category_column).distinct().filter(F.col(category_column).isNotNull()).count()),
        'numNulls_bestSalesCategory': int(df.filter(F.col(bestSalesCategory_column).isNull()).count()),
        'countDistinct_bestSalesCategory': int(df.select(bestSalesCategory_column).distinct().filter(F.col(bestSalesCategory_column).isNotNull()).count())
    }


    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_2')
    return res
    # -------------------------------------------------------------------------


In [10]:
res = task_2(data_io, data_dict['product'])
pa2.tests.test(res, 'task_2')

tests for task_2 --------------------------------------------------------------
Test 1/7 : countDistinct_bestSalesCategory ... Pass
Test 2/7 : countDistinct_category ... Pass
Test 3/7 : count_total ... Pass
Test 4/7 : mean_bestSalesRank ... Pass
Test 5/7 : numNulls_bestSalesCategory ... Pass
Test 6/7 : numNulls_category ... Pass
Test 7/7 : variance_bestSalesRank ... Pass
7/7 passed
-------------------------------------------------------------------------------


True

# Task 3





In [11]:
# %load -s task_3 assignment2.py
def task_3(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    asin_column = 'asin'
    price_column = 'price'
    attribute = 'also_viewed'
    related_column = 'related'
    # Outputs:
    meanPriceAlsoViewed_column = 'meanPriceAlsoViewed'
    countAlsoViewed_column = 'countAlsoViewed'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------
    
    # Explode also_viewed
    exploded = product_data.select(
        col(asin_column).alias('main_asin'),
        col(f'{related_column}.{attribute}').alias('related_asins')
    ).withColumn("related_asin", explode("related_asins"))

    # Join to get price of related products
    joined = exploded.join(
        product_data.select(col(asin_column).alias('related_asin'), col(price_column)),
        on='related_asin',
        how='left'
    )

    # Filter out rows where price is null
    valid_prices = joined.filter(col(price_column).isNotNull())

    # Compute meanPriceAlsoViewed per main product
    mean_price_df = valid_prices.groupBy('main_asin').agg(
        mean(price_column).alias(meanPriceAlsoViewed_column)
    )

    count_df = product_data.select(
        col(asin_column).alias('main_asin'),
        col(f'{related_column}.{attribute}').alias('related_asins')
    ).withColumn(
        countAlsoViewed_column,
        when(
            (col('related_asins').isNull()) | (size(col('related_asins')) == 0),
            None
        ).otherwise(size(col('related_asins')).cast(FloatType()))
    ).select('main_asin', countAlsoViewed_column)

    
    # Join mean and count
    summary_df = product_data.select(col(asin_column).alias('main_asin')) \
        .join(mean_price_df, on='main_asin', how='left') \
        .join(count_df, on='main_asin', how='left')


    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'mean_meanPriceAlsoViewed': None,
        'variance_meanPriceAlsoViewed': None,
        'numNulls_meanPriceAlsoViewed': None,
        'mean_countAlsoViewed': None,
        'variance_countAlsoViewed': None,
        'numNulls_countAlsoViewed': None
    }
    # Modify res:
    
    res = summary_df.agg(
        count("*").alias("count_total"),
        mean(meanPriceAlsoViewed_column).alias("mean_meanPriceAlsoViewed"),
        variance(meanPriceAlsoViewed_column).alias("variance_meanPriceAlsoViewed"),
        count(when(col(meanPriceAlsoViewed_column).isNull(), 1)).alias("numNulls_meanPriceAlsoViewed"),
        mean(countAlsoViewed_column).alias("mean_countAlsoViewed"),
        variance(countAlsoViewed_column).alias("variance_countAlsoViewed"),
        count(when(col(countAlsoViewed_column).isNull(), 1)).alias("numNulls_countAlsoViewed")
    ).first().asDict()


    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_3')
    return res
    # -------------------------------------------------------------------------


In [12]:
res = task_3(data_io, data_dict['product'])
pa2.tests.test(res, 'task_3')

tests for task_3 --------------------------------------------------------------
Test 1/7 : count_total ... Pass
Test 2/7 : mean_countAlsoViewed ... Pass
Test 3/7 : mean_meanPriceAlsoViewed ... Pass
Test 4/7 : numNulls_countAlsoViewed ... Pass
Test 5/7 : numNulls_meanPriceAlsoViewed ... Pass
Test 6/7 : variance_countAlsoViewed ... Pass
Test 7/7 : variance_meanPriceAlsoViewed ... Pass
7/7 passed
-------------------------------------------------------------------------------


True

# Task 4

In [13]:
# %load -s task_4 assignment2.py
def task_4(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    price_column = 'price'
    title_column = 'title'
    # Outputs:
    meanImputedPrice_column = 'meanImputedPrice'
    medianImputedPrice_column = 'medianImputedPrice'
    unknownImputedTitle_column = 'unknownImputedTitle'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------

    # Cast price to float
    product_data = product_data.withColumn(price_column, F.col(price_column).cast(T.FloatType()))

    # Compute mean and median of non-null prices
    price_stats = product_data.select(price_column).filter(F.col(price_column).isNotNull())
    price_mean = price_stats.agg(F.mean(price_column)).first()[0]
    price_median = price_stats.approxQuantile(price_column, [0.5], 0.01)[0]

    # Mean imputation
    with_mean = product_data.withColumn(
        meanImputedPrice_column,
        F.when(F.col(price_column).isNotNull(), F.col(price_column)).otherwise(F.lit(price_mean))
    )

    # Median imputation
    with_both = with_mean.withColumn(
        medianImputedPrice_column,
        F.when(F.col(price_column).isNotNull(), F.col(price_column)).otherwise(F.lit(price_median))
    )

    # Title imputation (nulls or empty strings set to 'unknown')
    with_all = with_both.withColumn(
        unknownImputedTitle_column,
        F.when(F.col(title_column).isNull() | (F.trim(F.col(title_column)) == ''), F.lit('unknown'))
         .otherwise(F.col(title_column))
    )


    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'mean_meanImputedPrice': None,
        'variance_meanImputedPrice': None,
        'numNulls_meanImputedPrice': None,
        'mean_medianImputedPrice': None,
        'variance_medianImputedPrice': None,
        'numNulls_medianImputedPrice': None,
        'numUnknowns_unknownImputedTitle': None
    }
    # Modify res:

    res = {
        'count_total': with_all.count(),
        'mean_meanImputedPrice': with_all.select(F.mean(meanImputedPrice_column)).first()[0],
        'variance_meanImputedPrice': with_all.select(F.variance(meanImputedPrice_column)).first()[0],
        'numNulls_meanImputedPrice': with_all.filter(F.col(meanImputedPrice_column).isNull()).count(),
        'mean_medianImputedPrice': with_all.select(F.mean(medianImputedPrice_column)).first()[0],
        'variance_medianImputedPrice': with_all.select(F.variance(medianImputedPrice_column)).first()[0],
        'numNulls_medianImputedPrice': with_all.filter(F.col(medianImputedPrice_column).isNull()).count(),
        'numUnknowns_unknownImputedTitle': with_all.filter(F.col(unknownImputedTitle_column) == 'unknown').count()
    }


    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_4')
    return res
    # -------------------------------------------------------------------------


In [14]:
res = task_4(data_io, data_dict['product'])
pa2.tests.test(res, 'task_4')

tests for task_4 --------------------------------------------------------------
Test 1/8 : count_total ... Pass
Test 2/8 : mean_meanImputedPrice ... Pass
Test 3/8 : mean_medianImputedPrice ... Pass
Test 4/8 : numNulls_meanImputedPrice ... Pass
Test 5/8 : numNulls_medianImputedPrice ... Pass
Test 6/8 : numUnknowns_unknownImputedTitle ... Pass
Test 7/8 : variance_meanImputedPrice ... Pass
Test 8/8 : variance_medianImputedPrice ... Pass
8/8 passed
-------------------------------------------------------------------------------


True

# Task 5

In [15]:
# %load -s task_5 assignment2.py
def task_5(data_io, product_processed_data, word_0, word_1, word_2):
    # -----------------------------Column names--------------------------------
    # Inputs:
    title_column = 'title'
    # Outputs:
    titleArray_column = 'titleArray'
    titleVector_column = 'titleVector'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------

    SEED = 102
    
    # Lowercase and split title
    product_processed_data_output = product_processed_data.withColumn(
        titleArray_column,
        F.split(F.lower(F.col(title_column)), ' ')
    )

    # Train Word2Vec
    word2vec = M.feature.Word2Vec(
        inputCol=titleArray_column,
        outputCol=titleVector_column,
        vectorSize=16,
        minCount=100,
        numPartitions=4,
        seed=SEED
    )
    model = word2vec.fit(product_processed_data_output)


    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'size_vocabulary': None,
        'word_0_synonyms': [(None, None), ],
        'word_1_synonyms': [(None, None), ],
        'word_2_synonyms': [(None, None), ]
    }
    # Modify res:
    res['count_total'] = product_processed_data_output.count()
    res['size_vocabulary'] = model.getVectors().count()
    for name, word in zip(
        ['word_0_synonyms', 'word_1_synonyms', 'word_2_synonyms'],
        [word_0, word_1, word_2]
    ):
        res[name] = model.findSynonymsArray(word, 10)
    
    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_5')
    return res
    # -------------------------------------------------------------------------


In [16]:
res = task_5(data_io, data_dict['product_processed'], 'piano', 'rice', 'laptop')
pa2.tests.test(res, 'task_5')

25/05/29 02:15:33 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/05/29 02:15:33 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
tests for task_5 --------------------------------------------------------------
Test 1/8 : count_total ... Pass
Test 2/8 : size_vocabulary ... Pass
Test 3/8 : word_0_synonyms-length ... Pass
Test 4/8 : word_0_synonyms-correctness ... Pass
Test 5/8 : word_1_synonyms-length ... Pass
Test 6/8 : word_1_synonyms-correctness ... Pass
Test 7/8 : word_2_synonyms-length ... Pass
Test 8/8 : word_2_synonyms-correctness ... Pass
5/5 passed
-------------------------------------------------------------------------------


True

# Task 6

In [17]:
# %load -s task_6 assignment2.py
def task_6(data_io, product_processed_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    category_column = 'category'
    # Outputs:
    categoryIndex_column = 'categoryIndex'
    categoryOneHot_column = 'categoryOneHot'
    categoryPCA_column = 'categoryPCA'
    # -------------------------------------------------------------------------    

    # ---------------------- Your implementation begins------------------------

    # StringIndexer
    indexer = StringIndexer(inputCol=category_column, outputCol=categoryIndex_column, handleInvalid='keep')
    indexed_data = indexer.fit(product_processed_data).transform(product_processed_data)

    # OneHotEncoder
    encoder = OneHotEncoder(inputCols=[categoryIndex_column],
                            outputCols=[categoryOneHot_column],
                            dropLast=True)
    encoded_data = encoder.fit(indexed_data).transform(indexed_data)

    # PCA
    pca = PCA(k=15, inputCol=categoryOneHot_column, outputCol=categoryPCA_column)
    pca_model = pca.fit(encoded_data)
    transformed_data = pca_model.transform(encoded_data)

    # Summarizer mean vectors 
    summarizer = Summarizer.metrics("mean")

    summary_onehot = transformed_data.select(Summarizer.mean(F.col(categoryOneHot_column)).alias("mean")).collect()[0]["mean"]
    summary_pca = transformed_data.select(Summarizer.mean(F.col(categoryPCA_column)).alias("mean")).collect()[0]["mean"]


    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'meanVector_categoryOneHot': [None, ],
        'meanVector_categoryPCA': [None, ]
    }
    # Modify res:

    res = {
        'count_total': transformed_data.count(),
        'meanVector_categoryOneHot': summary_onehot.toArray().tolist(),
        'meanVector_categoryPCA': summary_pca.toArray().tolist()
    }


    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_6')
    return res
    # -------------------------------------------------------------------------


In [18]:
res = task_6(data_io, data_dict['product_processed'])
pa2.tests.test(res, 'task_6')

25/05/29 02:15:56 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
25/05/29 02:15:56 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


tests for task_6 --------------------------------------------------------------
Test 1/9 : count_total ... Pass
Test 2/9 : meanVector_categoryOneHot-length ... Pass
Test 3/9 : meanVector_categoryOneHot-sum ... Pass
Test 4/9 : meanVector_categoryOneHot-mean ... Pass
Test 5/9 : meanVector_categoryOneHot-variance ... Pass
Test 6/9 : meanVector_categoryPCA-length ... Pass
Test 7/9 : meanVector_categoryPCA-sum ... Pass
Test 8/9 : meanVector_categoryPCA-mean ... Pass
Test 9/9 : meanVector_categoryPCA-variance ... Pass
9/9 passed
-------------------------------------------------------------------------------


True

In [19]:
print ("End to end time: {}".format(time.time()-begin))

End to end time: 686.8751242160797


# Part 2: Model Selection

In [20]:
# Bring the part_2 datasets to memory and de-cache part_1 datasets.
# Execute this once before you start working on this Part
data_dict, _ = data_io.cache_switch(data_dict, 'part_2')

# Task 7

In [21]:
def task_7(data_io, train_data, test_data):
    
    # ---------------------- Your implementation begins------------------------
    
    
    # Decision Tree Regressor with maxDepth=5
    dt = M.regression.DecisionTreeRegressor(featuresCol='features', labelCol='overall', maxDepth=5)

    # Fit model on training data
    model = dt.fit(train_data)

    # Predict on test data
    predictions = model.transform(test_data)

    # Evaluate using RMSE
    evaluator = M.evaluation.RegressionEvaluator(labelCol='overall', predictionCol='prediction', metricName='rmse')
    rmse = evaluator.evaluate(predictions)
    
    
    # -------------------------------------------------------------------------
    
    
    # ---------------------- Put results in res dict --------------------------
    res = {
        'test_rmse': None
    }
    # Modify res:

    res = {
        'test_rmse': rmse
    }

    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_7')
    return res
    # -------------------------------------------------------------------------

In [22]:
res = task_7(data_io, data_dict['ml_features_train'], data_dict['ml_features_test'])
pa2.tests.test(res, 'task_7')

tests for task_7 --------------------------------------------------------------
Test 1/1 : test_rmse ... Pass
1/1 passed
-------------------------------------------------------------------------------


True

# Task 8

In [23]:
def task_8(data_io, train_data, test_data):
    
    # ---------------------- Your implementation begins------------------------
    
    # Split original training data into new training and validation sets
    train_split, valid_split = train_data.randomSplit([0.75, 0.25], seed=102)

    # Define evaluator
    evaluator = M.evaluation.RegressionEvaluator(labelCol='overall', predictionCol='prediction', metricName='rmse')

    # Loop through each maxDepth value
    depths = [5, 7, 9, 12]
    valid_rmses = {}

    best_model = None
    best_depth = None
    best_rmse = float('inf')

    for depth in depths:
        dt = M.regression.DecisionTreeRegressor(featuresCol='features', labelCol='overall', maxDepth=depth)
        model = dt.fit(train_split)
        predictions = model.transform(valid_split)
        rmse = evaluator.evaluate(predictions)
        valid_rmses[depth] = rmse

        if rmse < best_rmse:
            best_rmse = rmse
            best_model = model
            best_depth = depth

    # Evaluate best model on test data
    test_predictions = best_model.transform(test_data)
    test_rmse = evaluator.evaluate(test_predictions)
    
    
    # -------------------------------------------------------------------------
    
    
    # ---------------------- Put results in res dict --------------------------
    res = {
        'test_rmse': None,
        'valid_rmse_depth_5': None,
        'valid_rmse_depth_7': None,
        'valid_rmse_depth_9': None,
        'valid_rmse_depth_12': None,
    }
    # Modify res:
    res = {
        'test_rmse': test_rmse,
        'valid_rmse_depth_5': valid_rmses[5],
        'valid_rmse_depth_7': valid_rmses[7],
        'valid_rmse_depth_9': valid_rmses[9],
        'valid_rmse_depth_12': valid_rmses[12],
    }

    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_8')
    return res
    # -------------------------------------------------------------------------

In [24]:
res = task_8(data_io, data_dict['ml_features_train'], data_dict['ml_features_test'])
pa2.tests.test(res, 'task_8')

tests for task_8 --------------------------------------------------------------
Test 1/5 : test_rmse ... Pass
Test 2/5 : valid_rmse_depth_12 ... Pass
Test 3/5 : valid_rmse_depth_5 ... Pass
Test 4/5 : valid_rmse_depth_7 ... Pass
Test 5/5 : valid_rmse_depth_9 ... Pass
5/5 passed
-------------------------------------------------------------------------------


True

In [25]:
print ("End to end time: {}".format(time.time()-begin))

End to end time: 1166.121743440628
